In [2]:
import sys, os
import numpy as np
from pyhdf.SD import SD
import pandas as pd
from datetime import datetime, time, timedelta
import re
import time
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import pearsonr, mode
import geopy
from matplotlib.patches import Ellipse
from functions import *
# Path and Variable declarations
# External harddrive path
file_ext= "D:\\NASA_AIRS"
file_path_plots=r"C:\Users\Zelda64\Documents\Programming\dust_solar_power\Plots"
# Internal testing folder
file_wind_2019=r"C:\Users\Zelda64\Documents\Programming\dust_solar_power\NREL_NSRDB\GHI_CAMS49_2019.csv"
file_wind_2020=r"C:\Users\Zelda64\Documents\Programming\dust_solar_power\NREL_NSRDB\GHI_CAMS49_2020.csv"
file_wind_2021=r"C:\Users\Zelda64\Documents\Programming\dust_solar_power\NREL_NSRDB\GHI_CAMS49_2021.csv"

#sort file list
folder_year_list=sorted(os.listdir(file_ext))

# Coordinates for TECQ stations
CAMS49 = [31.6676,-106.288]
CAMS1028 = [33.5856, -101.78]

In [30]:
wind_spd_dir_2019=pd.read_csv(file_wind_2019, header=[2], engine="python")
wind_spd_dir_2020=pd.read_csv(file_wind_2020, header=[2], engine="python")
wind_spd_dir_2021=pd.read_csv(file_wind_2021, header=[2], engine="python")

wind_spd_dir = pd.concat([wind_spd_dir_2019, wind_spd_dir_2020, wind_spd_dir_2021], ignore_index=True)

In [31]:
wind_spd_dir['Datetime']= pd.to_datetime(wind_spd_dir['Year'].astype(str) + '-' + wind_spd_dir['Month'].astype(str) + '-' + wind_spd_dir['Day'].astype(str) + '-' + wind_spd_dir['Hour'].astype(str), format='%Y-%m-%d-%H')
#datetime.strptime(datetimeobj,'%Y-%m-%d')
wind_spd_dir['Wind Speed']=wind_spd_dir['Wind Speed']*2.236936

In [32]:
wind_spd_dir.set_index('Datetime', inplace=True)
wind_spd_dir

,Year,Month,Day,Hour,Minute,Cloud Type,Wind Speed,Wind Direction,DHI,DNI,GHI,Solar Zenith Angle
Datetime,,,,,,,,,,,,
2019-01-01 00:00:00,2019,1,1,0,0,0,3.355404,118,0,0,0,171.15
2019-01-01 01:00:00,2019,1,1,1,0,0,2.908017,102,0,0,0,165.68
2019-01-01 02:00:00,2019,1,1,2,0,0,2.908017,92,0,0,0,153.87
2019-01-01 03:00:00,2019,1,1,3,0,0,3.131710,99,0,0,0,141.21
2019-01-01 04:00:00,2019,1,1,4,0,0,3.579098,108,0,0,0,128.46
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,2021,12,31,19,0,0,1.565855,290,0,0,0,112.28
2021-12-31 20:00:00,2021,12,31,20,0,0,2.013242,258,0,0,0,124.81
2021-12-31 21:00:00,2021,12,31,21,0,1,2.460630,243,0,0,0,137.54


In [33]:
def mode_float(data, num_bins):
    # Create histogram bins
    bins = np.linspace(np.min(data), np.max(data), num_bins)
    # Digitize the data into bins
    bin_indices = np.digitize(data, bins)
    # Find mode bin index
    mode_bin_index = mode(bin_indices).mode
    print(bin_indices)
    mode_value=data[mode_bin_index]
    # Calculate the mode value within the mode bin
    #mode_value = (bins[mode_bin_index] + bins[mode_bin_index + 1]) / 2
    
    # Count occurrences in the mode bin
    mode_frequency = np.sum(bin_indices == mode_bin_index)

    return mode_value, mode_frequency

def points_inside_ellipse(center_cams, center_windv, semi_major_axis, points):
    """Find points inside the given ellipse.
    center_cams is constant and based on which CAMS TCEQ ground station we want to use as a reference point
    center_windv is based on wind speed and direction and can take a position in a circle around center_cams
    while its distance depends on how far wind has carried dust in an hour

    
    """
    
    inside_points_dist = np.array([(distance(center_cams, point) + distance(center_windv, point)) for point in points])
    inside_points = points[inside_points_dist <= 2 * semi_major_axis]
    #inside_points = points[inside_points_dist <= semi_major_axis]
    '''
    for point in points:
        # Calculate the distance from each point to the two foci of the ellipse
        distance_to_foci = distance(center_cams, point) + distance(center_windv, point)
        # Check if the sum of distances is less than or equal to the major axis
        if distance_to_foci <= 2 * semi_major_axis:
            inside_points.append(point)
    '''
    return inside_points


def points_between_ellipses(first_focus, second_focus, first_focus_next, second_focus_next, semi_major_axis, semi_major_axis_next, points):
    inner_points = points_inside_ellipse(first_focus, second_focus, semi_major_axis, points)
    outer_points = points_inside_ellipse(first_focus_next, second_focus_next, semi_major_axis_next, points)
    between_points = np.array([point for point in outer_points if point not in inner_points])

    return between_points

In [3]:
# Create a dataframe for the 2019 to 2021 period for radial measurements
date_index = pd.date_range(start='2019-01-01', end='2021-12-31', freq='D')
time_of_day = pd.date_range(start='2019-01-01', periods=24, freq='H').time
dust_score_stats_CAMS49_mean = pd.DataFrame(index=date_index, columns=time_of_day)
dust_score_stats_CAMS49_median = dust_score_stats_CAMS49_mean
dust_score_stats_CAMS49_mode = dust_score_stats_CAMS49_mean


for year in folder_year_list:
    
    file_path_ext=os.path.join(file_ext,year)
    file_list=sorted(os.listdir(os.path.join(file_ext,file_path_ext)))
    print("Opening folder:", file_path_ext)

    
    file_name_cams49=f'CAMS49_{year}.xlsx'
    file_name_cams1028=f'CAMS1028_{year}.xlsx'
    tecq_cams49=read_xlsx_tecq(file_name_cams49)
    tecq_cams1028=read_xlsx_tecq(file_name_cams1028)

    last_ending_time_cams49=datetime(1900,1,1)

    for i, file_current in enumerate(file_list):
        if i < len(file_list) - 1:  # Ensure we don't go out of bounds
            file_next = file_list[i + 1]

        start_time=time.time()
        file_path=os.path.join(file_path_ext,file_current)
        file_path_next=os.path.join(file_path_ext,file_next)
        print("Opening file:", file_path)
        print("Next file: ", file_path_next)
        
        try:
            hdf_file = SD(file_path)
            # Select dust_score, lat and long     
            dust_score=hdf_file.select('dust_score')[:]
            long=hdf_file.select('Longitude')[:]
            lat=hdf_file.select('Latitude')[:]
            coords=coordinates(lat,long)
            print(dust_score.shape)
            # Find timestamp of production
            global_attributes = hdf_file.attributes()
            # Get raw productiondatetime from hdf file
            datetime_hdf_raw=find_rangedatetime(global_attributes)
            # Convert raw string into datetime object
            datetime_converted=datetime.strptime(datetime_hdf_raw,"%Y-%m-%dT%H:%M:%S.%fZ")
            # Round time from converted hdf datetime object to nearest hour 
            datetime_rounded_time=round_nearest_hour(datetime_converted)

                
            print("UTC: ",datetime_rounded_time)
            # Convert UTC to MST (only valid for CAMS49)
            datetime_rounded_time_mst=datetime_rounded_time-timedelta(hours=7)
            datetime_rounded_time_cst=datetime_rounded_time-timedelta(hours=6)
            print("MST: ",datetime_rounded_time_mst)
            print("CST: ",datetime_rounded_time_cst)

            # Convert datetime object (date part only) into string
            datetime_converted_str_mst=datetime_rounded_time_mst.date().strftime("%Y-%m-%d")
            datetime_converted_str_cst=datetime_rounded_time_cst.date().strftime("%Y-%m-%d")

            '''
            Load next hdf file to find the datetime of its data
            '''
            hdf_file_next = SD(file_path_next)
            # Select dust_score, lat and long     
            global_attributes_next = hdf_file_next.attributes()
            # Get raw productiondatetime from hdf file
            datetime_hdf_raw_next=find_rangedatetime(global_attributes_next)
            # Convert raw string into datetime object
            datetime_converted_next=datetime.strptime(datetime_hdf_raw_next,"%Y-%m-%dT%H:%M:%S.%fZ")
            # Round time from converted hdf datetime object to nearest hour 
            datetime_rounded_time_next=round_nearest_hour(datetime_converted_next)
            next_datetime_rounded_time_mst=datetime_rounded_time_next-timedelta(hours=7)
            hdf_file_next.end()
            
            perhour =1


            
            #for radius=0 find mean/mode/median as close as possible to cams station
            coords_reshaped=np.reshape(coords,(coords.shape[0]*coords.shape[1],2))
            observation_starting_time_cams49=datetime_rounded_time_mst
            print("Time of satellite data capture: ", observation_starting_time_cams49)

            avg_windspd_cams49=wind_spd_dir.at[datetime_rounded_time_mst.strftime('%Y-%m-%d-%H'),'Wind Speed']
            print("Wind speed at start of AIRS recording: ",avg_windspd_cams49)

            first_focus_last=0
            second_focus_last=0
            semi_major_axis_last=0
            for elem in range(len(wind_spd_dir)):
                #Wind speed on certain day of the year as measured at El Paso airport, only 2019 data so far!
                
                avg_winddir_cams49=wind_spd_dir.at[observation_starting_time_cams49.strftime('%Y-%m-%d-%H'),'Wind Direction']
                if avg_winddir_cams49<180:
                    avg_winddir_cams49=int((180+avg_winddir_cams49))
                else:
                    avg_winddir_cams49=int(np.abs(180-avg_winddir_cams49))            
                #Draw circle every 2 hours for 12 hours, until next satellite obesrvation (which could be on the same or the next day)
                #How much has the dust moved within the 1st two hours, look around at a circle of that radius, where the CAMS49 is a vertex on the cirle
                if observation_starting_time_cams49>=next_datetime_rounded_time_mst:
                    print("Warning: datetime overlap! ",observation_starting_time_cams49, next_datetime_rounded_time_mst)
                    break
                   
                #find the other vertex based on the distance wind travels in an hour/2hours
                distance_vertices=avg_windspd_cams49
                distance_to_outer_vertex=avg_windspd_cams49+wind_spd_dir.at[(observation_starting_time_cams49+timedelta(hours=perhour)).strftime('%Y-%m-%d-%H'),'Wind Speed']
                degrees_change=geopy.units.degrees(arcminutes=geopy.units.nautical(miles=distance_vertices))
                degrees_change_next=geopy.units.degrees(arcminutes=geopy.units.nautical(miles=distance_to_outer_vertex))
                #finding the other vertex of the same ellipse
                lat_vertex=CAMS49[0]+degrees_change*np.sin(np.radians(avg_winddir_cams49))
                long_vertex=CAMS49[1]+degrees_change*np.cos(np.radians(avg_winddir_cams49))
                vertex_windv= (lat_vertex, long_vertex)
                
                #finding the vertex of the ellipse in the next loop
                lat_next=CAMS49[0]+degrees_change_next*np.sin(np.radians(avg_winddir_cams49))
                long_next=CAMS49[1]+degrees_change_next*np.cos(np.radians(avg_winddir_cams49))
                next_vertex_windv= (lat_next, long_next)
                
                eccentricity=0.7
                lat_center=CAMS49[0]+degrees_change/2*np.sin(np.radians(avg_winddir_cams49))
                long_center=CAMS49[1]+degrees_change/2*np.cos(np.radians(avg_winddir_cams49))
                ellipse_center=[lat_center,long_center]
                    #dist_center=distance(CAMS49,[lat_center,long_center])
                    #print(dist_center)
                    #semi_major_axis is windspd_radii_cams49[elem]
                semi_major_axis=distance_vertices/2
                    #Currently minor axis is useless
                semi_minor_axis= semi_major_axis*np.sqrt(1 - (eccentricity**2))
                dist_center_to_foci=semi_major_axis*eccentricity
                    #dist_center_to_foci=semi_major_axis**2-(semi_minor_axis)**2
                    #print("DIST to CENTER")
                    #print(dist_center_to_foci,dist_center_to_foci2)
                first_focus=[lat_center+geopy.units.degrees(arcminutes=geopy.units.nautical(miles=dist_center_to_foci))*np.sin(np.radians(avg_winddir_cams49)-np.pi),
                            long_center+geopy.units.degrees(arcminutes=geopy.units.nautical(miles=dist_center_to_foci))*np.cos(np.radians(avg_winddir_cams49)-np.pi)]
                second_focus=[lat_center+geopy.units.degrees(arcminutes=geopy.units.nautical(miles=dist_center_to_foci))*np.sin(np.radians(avg_winddir_cams49)),
                            long_center+geopy.units.degrees(arcminutes=geopy.units.nautical(miles=dist_center_to_foci))*np.cos(np.radians(avg_winddir_cams49))]
    
    
                #FIND FOCI OF NEXT ELLIPSE
                eccentricity_next=eccentricity#+0.2
                lat_center_next=CAMS49[0]+degrees_change_next/2*np.sin(np.radians(avg_winddir_cams49))
                long_center_next=CAMS49[1]+degrees_change_next/2*np.cos(np.radians(avg_winddir_cams49))
    

                semi_major_axis_next=distance_to_outer_vertex/2
                dist_center_to_foci_next=semi_major_axis*eccentricity
    
                    
                first_focus_next=[lat_center_next+geopy.units.degrees(arcminutes=geopy.units.nautical(miles=dist_center_to_foci_next))*np.sin(np.radians(avg_winddir_cams49)-np.pi),
                            long_center_next+geopy.units.degrees(arcminutes=geopy.units.nautical(miles=dist_center_to_foci_next))*np.cos(np.radians(avg_winddir_cams49)-np.pi)]
                second_focus_next=[lat_center_next+geopy.units.degrees(arcminutes=geopy.units.nautical(miles=dist_center_to_foci_next))*np.sin(np.radians(avg_winddir_cams49)),
                            long_center_next+geopy.units.degrees(arcminutes=geopy.units.nautical(miles=dist_center_to_foci_next))*np.cos(np.radians(avg_winddir_cams49))]                
                
                inside_points=points_inside_ellipse(first_focus, second_focus, semi_major_axis, coords_reshaped)    
                if len(inside_points)!=0:
                    if elem==0:
                        print("Points found inside first ellipse, size: ", inside_points)
                        matched_circle_coords=match_coords_circle(coords, inside_points)
                        
                        dust_score_mean_between_points=np.mean(dust_score[matched_circle_coords])
                        dust_score_median_between_points=np.median(dust_score[matched_circle_coords])
                        dust_score_mode_between_points=mode_float(dust_score[matched_circle_coords],int(len(dust_score[matched_circle_coords]))-1)[0]
                        dust_score_stats_CAMS49_mean.at[observation_starting_time_cams49.strftime('%Y-%m-%d'), observation_starting_time_cams49.time()]=dust_score_mean_between_points
                        dust_score_stats_CAMS49_median.at[observation_starting_time_cams49.strftime('%Y-%m-%d'), observation_starting_time_cams49.time()]=dust_score_median_between_points
                        dust_score_stats_CAMS49_mode.at[observation_starting_time_cams49.strftime('%Y-%m-%d'), observation_starting_time_cams49.time()]=dust_score_mode_between_points
                        '''
                        draw_grid_plot(CAMS49, coords, inside_points, file_current, observation_starting_time_cams49.strftime('%Y-%m-%d  %H:%M:%S'), dust_score, round(distance_vertices,2), 
                                       True, round(wind_spd_dir.at[observation_starting_time_cams49.strftime('%Y-%m-%d-%H'),'Wind Speed'],2), avg_winddir_cams49, vertex_windv, semi_major_axis, semi_minor_axis)
                        '''
                        observation_starting_time_cams49+=timedelta(hours=perhour)
                        avg_windspd_cams49+=wind_spd_dir.at[observation_starting_time_cams49.strftime('%Y-%m-%d-%H'),'Wind Speed']
                        first_focus_last=first_focus
                        second_focus_last=second_focus
                        semi_major_axis_last=semi_major_axis
                        continue
                    # Points between the circles
                    between_points=points_between_ellipses(first_focus_last, second_focus_last, first_focus_next,second_focus_next, semi_major_axis_last, semi_major_axis_next, coords_reshaped)
                                        
                    if len(between_points)!=0:
                        matched_circle_coords=match_coords_circle(coords, between_points)
                        
                        print("Points found between the two ellipses, size: ", between_points.shape)

                        '''
                        draw_grid_plot(CAMS49, coords, inside_points, file_current, observation_starting_time_cams49.strftime('%Y-%m-%d  %H:%M:%S'), dust_score, round(distance_to_outer_vertex,2), 
                                   True,round(wind_spd_dir.at[observation_starting_time_cams49.strftime('%Y-%m-%d-%H'),'Wind Speed'],2), avg_winddir_cams49, vertex_windv, semi_major_axis, semi_minor_axis, between_points)
                        '''
                        dust_score_mean_between_points=np.mean(dust_score[matched_circle_coords])
                        dust_score_median_between_points=np.median(dust_score[matched_circle_coords])
                        dust_score_mode_between_points=mode_float(dust_score[matched_circle_coords],int(len(dust_score[matched_circle_coords]))-1)[0]
                        
                        dust_score_stats_CAMS49_mean.at[observation_starting_time_cams49.strftime('%Y-%m-%d'), observation_starting_time_cams49.time()]=dust_score_mean_between_points
                        dust_score_stats_CAMS49_median.at[observation_starting_time_cams49.strftime('%Y-%m-%d'), observation_starting_time_cams49.time()]=dust_score_median_between_points
                        dust_score_stats_CAMS49_mode.at[observation_starting_time_cams49.strftime('%Y-%m-%d'), observation_starting_time_cams49.time()]=dust_score_mode_between_points

                        #update the last_ending_time with the latest time written into the dataframe
                        last_ending_time_cams49=observation_starting_time_cams49
                    
                else:
                    print("No points found inside first ellipse, drawing next circle...")
                    
                observation_starting_time_cams49+=timedelta(hours=perhour)
                avg_windspd_cams49+=wind_spd_dir.at[observation_starting_time_cams49.strftime('%Y-%m-%d-%H'),'Wind Speed']
                first_focus_last=first_focus
                second_focus_last=second_focus
                semi_major_axis_last=semi_major_axis
                print(avg_windspd_cams49)
            print("Done")

            # Close the HDF4 file
            hdf_file.end()
        except Exception as e:
            print("Error opening file:", file_path)
            print(e)
            print("\n")
        end_time=time.time()
        runtime = end_time - start_time
        print(f"HDF file execution time: {runtime} sec")
        print("\n")
    
        
    
'''
if __name__ == "__main__":
'''

Opening folder: D:\NASA_AIRS\2019
Opening file: D:\NASA_AIRS\2019\AIRS.2019.01.01.082.L1B.AIRS_Rad.v5.0.23.0.G19001113504.hdf
Next file:  D:\NASA_AIRS\2019\AIRS.2019.01.01.098.L1B.AIRS_Rad.v5.0.23.0.G19001113440.hdf
(135, 90)
UTC:  2019-01-01 08:00:00
MST:  2019-01-01 01:00:00
CST:  2019-01-01 02:00:00
Time of satellite data capture:  2019-01-01 01:00:00
Error opening file: D:\NASA_AIRS\2019\AIRS.2019.01.01.082.L1B.AIRS_Rad.v5.0.23.0.G19001113504.hdf
name 'wind_spd_dir' is not defined


HDF file execution time: 3.0201869010925293 sec


Opening file: D:\NASA_AIRS\2019\AIRS.2019.01.01.098.L1B.AIRS_Rad.v5.0.23.0.G19001113440.hdf
Next file:  D:\NASA_AIRS\2019\AIRS.2019.01.01.192.L1B.AIRS_Rad.v5.0.23.0.G19002100753.hdf
(135, 90)
UTC:  2019-01-01 10:00:00
MST:  2019-01-01 03:00:00
CST:  2019-01-01 04:00:00
Time of satellite data capture:  2019-01-01 03:00:00
Error opening file: D:\NASA_AIRS\2019\AIRS.2019.01.01.098.L1B.AIRS_Rad.v5.0.23.0.G19001113440.hdf
name 'wind_spd_dir' is not defined





KeyboardInterrupt



In [38]:
#dust_score_stats_CAMS49_mean.to_csv(r'C:\Users\Zelda64\Documents\Programming\dust_solar_power\dust_score_stats_CAMS49ELLIPSE_mean_hourly_new.csv', encoding='utf-8')
dust_score_stats_CAMS49_mode.to_csv(r'C:\Users\Zelda64\Documents\Programming\dust_solar_power\dust_score_stats_CAMS49ELLIPSE_mode_hourly_new.csv', encoding='utf-8')
dust_score_stats_CAMS49_median.to_csv(r'C:\Users\Zelda64\Documents\Programming\dust_solar_power\dust_score_stats_CAMS49ELLIPSE_median_hourly_new.csv', encoding='utf-8')

In [ ]:
'''
def plot_dust_pm(type: str, threshold_pm: int, threshold_dust_score: int, dust_score_pm_matches_array: list, additional_array: list = None):
    index_year_2019=dust_score_pm_matches_array.index(['2019', '2019'])
    index_year_2020=dust_score_pm_matches_array.index(['2020', '2020'])

    dust_score_pm_matches_2019=np.array(dust_score_pm_matches_array[index_year_2019+1:index_year_2020])
    dust_score_pm_matches_2020=np.array(dust_score_pm_matches_array[index_year_2020+1:])

    if additional_array is not None:
        
        index_year_2019_add=additional_array.index(['2019', '2019'])
        index_year_2020_add=additional_array.index(['2020', '2020'])
        additional_array_2019=np.array(additional_array[index_year_2019_add+1:index_year_2020_add])
        additional_array_2020=np.array(additional_array[index_year_2020_add+1:]) 
        dust_score_pm_matches_2019=np.concatenate((dust_score_pm_matches_2019,additional_array_2019), axis=0)
        dust_score_pm_matches_2020=np.concatenate((dust_score_pm_matches_2020,additional_array_2020), axis=0)

    mask_matches_2019=np.isnan(dust_score_pm_matches_2019).any(axis=1)
    mask_matches_2020=np.isnan(dust_score_pm_matches_2020).any(axis=1)
    
    dust_score_pm_matches_2019=dust_score_pm_matches_2019[~mask_matches_2019]
    dust_score_pm_matches_2020=dust_score_pm_matches_2020[~mask_matches_2020]
    
    dust_score_pm_matches_2020_filtered=dust_score_pm_matches_2020[np.logical_and(dust_score_pm_matches_2020[:,0]>threshold_pm, dust_score_pm_matches_2020[:,1]>threshold_dust_score)]
    dust_score_pm_matches_2019_filtered=dust_score_pm_matches_2019[np.logical_and(dust_score_pm_matches_2019[:,0]>threshold_pm, dust_score_pm_matches_2019[:,1]>threshold_dust_score)]
    print(dust_score_pm_matches_2019_filtered.shape)
    print(dust_score_pm_matches_2020_filtered.shape)
    

    x_pm_2019=dust_score_pm_matches_2019_filtered[:,0]
    y_dust_score_2019=dust_score_pm_matches_2019_filtered[:,1]
    
    x_pm_2020=dust_score_pm_matches_2020_filtered[:,0]
    y_dust_score_2020=dust_score_pm_matches_2020_filtered[:,1]
    
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(1, 1, 1)
    
    ax.scatter(x_pm_2019,y_dust_score_2019, marker='.', label='2019')
    ax.scatter(x_pm_2020,y_dust_score_2020, marker='.', label='2020')
        
    # Add title and show the plot
    plt.title(f'{type} PM from CAMS49/1028 plotted with AIRS Dust_score')
    plt.xlabel("PM2.5 (µg/m3)")
    plt.ylabel("Dust_score")
    plt.legend()
    plt.show()

    corr_coefficient_2019, p_value_2019 = pearsonr(x_pm_2019, y_dust_score_2019)
    corr_coefficient_2020, p_value_2020 = pearsonr(x_pm_2020, y_dust_score_2020)
    corr_coefficient_comb, p_vale_comb = pearsonr(np.concatenate((x_pm_2019,x_pm_2020)), np.concatenate((y_dust_score_2019,y_dust_score_2020)))
    print("Pearson correlation coefficient 2019 data:", corr_coefficient_2019.round(3))
    print("p-value:", p_value_2019.round(3))
    print("Pearson correlation coefficient 2020 data:", corr_coefficient_2020.round(3))
    print("p-value:", p_value_2020.round(3))
    print("Pearson correlation coefficient 2019 and 2020 data:", corr_coefficient_comb.round(3))
    print("p-value:", p_vale_comb.round(3))

    plt.hist(x_pm_2019, alpha=0.5, label='PM2.5 2019')
    plt.hist(x_pm_2020, alpha=0.5, label='PM2.5 2020')
    plt.hist(y_dust_score_2019, alpha=0.5, label='Dust_score 2019')
    plt.hist(y_dust_score_2020, alpha=0.5, label='Dust_score 2020')
    
    plt.xlabel('PM2.5 & Dust_score Values')
    plt.ylabel('Frequency')
    plt.title('Max PM2.5 & Dust_score Data Distribution')
    
    plt.legend()
        
plot_dust_pm('Mean', 20, 0,dust_score_pm_matches_cams1028,dust_score_pm_matches_cams49)

'''